In [15]:
import os
import requests
import json 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas import json_normalize
import time
from defillama import DefiLlama
from datetime import datetime
llama = DefiLlama()
import datetime as dt
import pytz


In [ ]:
chains = ['Ethereum','Solana','Base','Sei','Sui','Injective','Avalanche','Optimism']

In [82]:
def get_net_flows(chain_name:str):
    ids = []
    aggregated_data = {}
    change_in_flows = {'24H': 0, '7D': 0, '30D': 0, '60D':0}

    bridges = "https://bridges.llama.fi/bridges?includeChains=true"
    bridges = requests.get(bridges).json()
    
    for i in range(len(bridges['bridges'])):
        if f"{chain_name}" in bridges['bridges'][i]['chains']:
            ids.append(bridges['bridges'][i]['id'])
    
    for id in ids:
        url = f"https://bridges.llama.fi/bridgevolume/{chain_name}?id={id}"
        vol_data = requests.get(url).json()[-60:]

        for entry in vol_data:
            date = datetime.utcfromtimestamp(int(entry['date']))
            eastern = pytz.timezone("US/Eastern")
            date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
            date = date.strftime('%Y-%m-%d')
            net_inflow = entry['depositUSD'] - entry['withdrawUSD']

            if date in aggregated_data:
                aggregated_data[date] += net_inflow
            else:
                aggregated_data[date] = net_inflow
    sorted_flows = sorted(aggregated_data.items(), key=lambda x: x[0])

    one_day_net_flow = round(sorted_flows[-1][-1],2)
    seven_day_net_flow = round(sum(flow[1] for flow in sorted_flows[-7:]),2)
    thirty_day_net_flow = round(sum(flow[1] for flow in sorted_flows[-30:]),2)
    sixty_day_net_flow = round(sum(flow[1] for flow in sorted_flows[-60:]),2)

    change_in_flows['24H'] = one_day_net_flow
    change_in_flows['7D'] = seven_day_net_flow
    change_in_flows['30D'] = thirty_day_net_flow
    change_in_flows['60D'] = sixty_day_net_flow

    return sorted_flows, change_in_flows

In [97]:
def get_stables(chain_name:str):
    stable_ids = []
    aggregated_stables = {}
    change_in_stables = {'24H': 0, '24H % Chg':0, '7D': 0, '7D % Chg':0, '30D': 0, '30D % Chg':0, '60D':0, '60D % Chg':0}

    stables_url = "https://stablecoins.llama.fi/stablecoins?includePrices=true"
    stable_id = requests.get(stables_url).json()

    for i in range(len(stable_id['peggedAssets'])):
        stable_ids.append(stable_id['peggedAssets'][i]['id'])
    
    
    for id in stable_ids:
        url = F"https://stablecoins.llama.fi/stablecoincharts/{chain_name}?stablecoin={id}"
       
        stables_data = requests.get(url).json()[-60:]
        if len(stables_data) > 0:
       
            for entry in stables_data:
                date = datetime.utcfromtimestamp(int(entry['date']))
                eastern = pytz.timezone("US/Eastern")
                date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
                date = date.strftime('%Y-%m-%d')

                if date in aggregated_stables:
                    aggregated_stables[date] += entry['totalCirculating']['peggedUSD']
                else:
                    aggregated_stables[date] = entry['totalCirculating']['peggedUSD']
        else:
            pass
    sorted_agg_stables = sorted(aggregated_stables.items(), key = lambda x:x[0])

    
    one_day_stables = round((sorted_agg_stables[-1][-1]-sorted_agg_stables[-2][-1]),2)
    seven_day_stables = round((sorted_agg_stables[-1][-1]-sorted_agg_stables[-7][-1]),2)
    thirty_day_stables = round((sorted_agg_stables[-1][-1]-sorted_agg_stables[-30][-1]),2)
    sixty_day_stables = round((sorted_agg_stables[-1][-1]-sorted_agg_stables[-60][-1]),2)
    one_day_pct_chg = round(((sorted_agg_stables[-1][-1]/sorted_agg_stables[-2][-1])-1) * 100,3)
    seven_day_pct_chg = round(((sorted_agg_stables[-1][-1]/sorted_agg_stables[-8][-1])-1) * 100,3)
    thirty_day_pct_chg = round(((sorted_agg_stables[-1][-1]/sorted_agg_stables[-31][-1])-1) * 100,3)
    sixty_day_pct_chg = round(((sorted_agg_stables[-1][-1]/sorted_agg_stables[-60][-1])-1) * 100,3)

    change_in_stables['24H'] = one_day_stables
    change_in_stables['7D'] = seven_day_stables
    change_in_stables['30D'] = thirty_day_stables
    change_in_stables['60D'] = sixty_day_stables
    change_in_stables['24H % Chg'] = one_day_pct_chg
    change_in_stables['7D % Chg'] = seven_day_pct_chg
    change_in_stables['30D % Chg'] = thirty_day_pct_chg
    change_in_stables['60D % Chg'] = sixty_day_pct_chg


    return sorted_agg_stables, change_in_stables
    

In [99]:
def get_tvl(chain_name:str):
    tvl = {}
    change_in_tvl = {'24H': 0, '24H % Chg':0, '7D': 0, '7D % Chg':0, '30D': 0, '30D % Chg':0, '60D':0, '60D % Chg':0}
    tvl_url = F"https://api.llama.fi/v2/historicalChainTvl/{chain_name}"
    tvl_data = requests.get(tvl_url).json()[-60:]

    for entry in tvl_data:
        date = datetime.utcfromtimestamp(int(entry['date']))
        eastern = pytz.timezone("US/Eastern")
        date = date.replace(tzinfo=pytz.utc).astimezone(eastern)
        date = date.strftime('%Y-%m-%d')

        
        tvl[date] = entry['tvl']
    
    sorted_tvl = sorted(tvl.items(), key = lambda x:x[0])

    one_day_tvl = round((sorted_tvl[-1][-1]-sorted_tvl[-2][-1]),2)
    seven_day_tvl = round((sorted_tvl[-1][-1]-sorted_tvl[-7][-1]),2)
    thirty_day_tvl = round((sorted_tvl[-1][-1]-sorted_tvl[-30][-1]),2)
    sixty_day_tvl = round((sorted_tvl[-1][-1]-sorted_tvl[-60][-1]),2)
    one_day_pct_chg = round(((sorted_tvl[-1][-1]/sorted_tvl[-2][-1])-1) * 100,3)
    seven_day_pct_chg = round(((sorted_tvl[-1][-1]/sorted_tvl[-8][-1])-1) * 100,3)
    thirty_day_pct_chg = round(((sorted_tvl[-1][-1]/sorted_tvl[-31][-1])-1) * 100,3)
    sixty_day_pct_chg = round(((sorted_tvl[-1][-1]/sorted_tvl[-60][-1])-1) * 100,3)

    change_in_tvl['24H'] = one_day_tvl
    change_in_tvl['7D'] = seven_day_tvl
    change_in_tvl['30D'] = thirty_day_tvl
    change_in_tvl['60D'] = sixty_day_tvl
    change_in_tvl['24H % Chg'] = one_day_pct_chg
    change_in_tvl['7D % Chg'] = seven_day_pct_chg
    change_in_tvl['30D % Chg'] = thirty_day_pct_chg
    change_in_tvl['60D % Chg'] = sixty_day_pct_chg

    return sorted_tvl, change_in_tvl

In [66]:
def get_dex_vol(chain_name:str):
    dex_volume = {'24H': 0, '7D': 0, '30D': 0}
    dex_url = f"https://api.llama.fi/overview/dexs/{chain_name}?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"

    try:
        dex_data = requests.get(dex_url).json()
        for entry in dex_data['protocols']:
            dex_volume['24H'] += entry.get('total24h', 0)
            dex_volume['7D'] += entry.get('total7d', 0)
            dex_volume['30D'] += entry.get('total30d', 0)
    except Exception as e:
        print(f"An error occurred: {e}")

    return dex_volume

In [83]:
# Works perfect
flows, chg = get_net_flows("Base")
flows, chg

[('2023-08-23', -14200),
 ('2023-08-24', -18300),
 ('2023-09-02', 22493),
 ('2023-09-03', 20391),
 ('2023-09-04', 18875),
 ('2023-09-05', -139561),
 ('2023-09-06', 35168),
 ('2023-09-07', 14966),
 ('2023-09-08', 2163),
 ('2023-09-09', 0),
 ('2023-09-10', -44252),
 ('2023-09-11', 0),
 ('2023-12-13', -880214),
 ('2023-12-14', 678564),
 ('2023-12-15', -1223),
 ('2023-12-16', -389645),
 ('2023-12-17', 1441277),
 ('2023-12-18', -16527),
 ('2023-12-19', 420747),
 ('2023-12-20', -123325),
 ('2023-12-21', 360014),
 ('2023-12-22', -20725),
 ('2023-12-23', -347989),
 ('2023-12-24', -1228460),
 ('2023-12-25', 1461363),
 ('2023-12-26', 2695495),
 ('2023-12-27', 783229),
 ('2023-12-28', -10166),
 ('2023-12-29', -2061385),
 ('2023-12-30', 1173834),
 ('2023-12-31', 1308024),
 ('2024-01-01', 1474411),
 ('2024-01-02', 1813352),
 ('2024-01-03', 3830154),
 ('2024-01-04', 1374970),
 ('2024-01-05', 1244989),
 ('2024-01-06', 803171),
 ('2024-01-07', 2613156),
 ('2024-01-08', 1680069),
 ('2024-01-09', 389153

In [98]:
# works perfect
stables, stbl_chg = get_stables("Base")
stables, stbl_chg

([('2023-12-13', 107922269.14),
  ('2023-12-14', 113661162.69999999),
  ('2023-12-15', 114043439.00999999),
  ('2023-12-16', 115490274.09),
  ('2023-12-17', 119221047.35999998),
  ('2023-12-18', 132993824.1),
  ('2023-12-19', 314861065.9699999),
  ('2023-12-20', 316388133.04),
  ('2023-12-21', 316970270.52),
  ('2023-12-22', 315986949.27000004),
  ('2023-12-23', 315565860.55999994),
  ('2023-12-24', 315262875.35),
  ('2023-12-25', 317749103.34999996),
  ('2023-12-26', 275951190.32),
  ('2023-12-27', 218537090.82),
  ('2023-12-28', 187326849.14),
  ('2023-12-29', 175378500.05999997),
  ('2023-12-30', 177024479.71),
  ('2023-12-31', 175325066.05),
  ('2024-01-01', 177669277.85999998),
  ('2024-01-02', 184579757.69),
  ('2024-01-03', 314266286.34),
  ('2024-01-04', 317839237.6499999),
  ('2024-01-05', 319590748.8000001),
  ('2024-01-06', 317033760.56),
  ('2024-01-07', 314139373.32000005),
  ('2024-01-08', 307864128.15000004),
  ('2024-01-09', 306009390.9299999),
  ('2024-01-10', 30772288

In [100]:
# Works perfect
tvl, chg_tvl = get_tvl('Base')
tvl, chg_tvl

([('2023-12-13', 387152661.7283448),
  ('2023-12-14', 397666121.8108578),
  ('2023-12-15', 399264419.43908405),
  ('2023-12-16', 405678835.8462741),
  ('2023-12-17', 402537287.8680279),
  ('2023-12-18', 419396423.2817039),
  ('2023-12-19', 419343714.013779),
  ('2023-12-20', 443597658.20717096),
  ('2023-12-21', 450589267.63081485),
  ('2023-12-22', 448162223.74464345),
  ('2023-12-23', 451896696.9431839),
  ('2023-12-24', 452369902.736831),
  ('2023-12-25', 451713120.5597483),
  ('2023-12-26', 453029852.83770305),
  ('2023-12-27', 454395325.1113004),
  ('2023-12-28', 451576125.90438926),
  ('2023-12-29', 440119845.03400195),
  ('2023-12-30', 441349827.08224756),
  ('2023-12-31', 437157255.99282956),
  ('2024-01-01', 444171067.9471625),
  ('2024-01-02', 438070813.3351467),
  ('2024-01-03', 420751925.6382915),
  ('2024-01-04', 421371948.62472284),
  ('2024-01-05', 418966482.70274866),
  ('2024-01-06', 418422993.3203411),
  ('2024-01-07', 412480553.28625035),
  ('2024-01-08', 414988142.1

In [67]:
# Works perfect
dex_volume = get_dex_vol('Base')
dex_volume

{'24H': 46886056.82357019, '7D': 279456085.21509826, '30D': 828820277.4673687}